In [39]:
import numpy as np
import astropy.io.fits as fits
import pandas as pd
import math
pd.set_option('display.max_colwidth', None)


In [2]:
with fits.open('../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits') as F:
    print(F.info())
    print(F[0].header.cards)
    objid=F[1].data['objid']
    ps_score=F[1].data['ps_score']

Filename: ../ps1_psc/hlsp_ps1-psc_ps1_gpc1_0_multi_v1_cat.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   ()      
  1                1 BinTableHDU     14   16682163R x 2C   [K, E]   
None
('SIMPLE', True, 'conforms to FITS standard')
('BITPIX', 8, 'array data type')
('NAXIS', 0, 'number of array dimensions')
('EXTEND', True, '')
('HLSPTARG', '0', '')
('INSTRUME', 'PS1 survey', '')
('OBSERVAT', 'Haleakala Observatories', '')
('TELESCOP', 'Pan-STARRS1 Telescope', '')
('HLSPID', 'ps1_psc', '')
('HLSPLEAD', 'Adam A Miller', '')
('HLSPNAME', 'Probabilistic Classifications of Unresolved Point Sources in PanSTARRS1', '')
('HLSPVER', 'v1', '')
('LICENSE', 'CC BY 4.0', '')
('LICENURL', 'https://creativecommons.org/licenses/by/4.0/', '')
('REFERENC', 'https://doi.org/10.1088/1538-3873/aae3d9', '')
('HLSPDOI', 'https://doi.org/10.17909/t9-xjrf-7g34', '')


In [3]:
len(objid)

16682163

In [20]:
objid

array([108082954545197165, 108042899536471576, 108173097098982498, ...,
       108862811948172319, 109012792577601116, 108910340111860145])

In [30]:
ps_score[np.where(objid == 108082954545197165)[0][0]]

0.5813869

In [5]:
# score of 0 corresponding to extended objects and 1 corresponding to point sources
# extended objects are resolved galaxies
# point sources are unresolved stars or QSOs

In [6]:
len(ps_score)

16682163

In [7]:
ps_score

array([0.5813869, 0.8385   , 0.99875  , ..., 0.5611875, 0.9922917,
       0.       ], dtype=float32)

In [8]:
%%time

# Instead of above, read in from csv files with host gal info!
# Visually vetted GHOST results split up over XXX files
# So merge them

dfs = []
for start_idx, end_idx in zip(range(0, 90100, 100), range(100, 90200, 100)): #90199, 90299
    df = pd.read_csv(f'./host_info/dataset_bank_hosts_start_idx:end_idx={start_idx}:{end_idx}.csv')
    dfs.append(df)
    
g_hosts_df = pd.concat(dfs, axis=0).reset_index(drop=True)
g_hosts_df.drop_duplicates(subset="TransientName", keep='first', inplace=True)
g_hosts_df = g_hosts_df.set_index('TransientName')
g_hosts_df.sort_values('TransientName', inplace=True)
#g_hosts_df.reset_index(inplace=True, drop=True)
g_hosts_df

CPU times: user 17.6 s, sys: 1.02 s, total: 18.6 s
Wall time: 19.9 s


,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
TransientName,,,,,,,,,,,,,,,,,,,,,
AT2019ye,PSO J135804.487+203809.056,NaN,NaN,NaN,132762095186853570,916270000024592,143378893266959,0,14931762990582,13276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PS18acw,PSO J203708.962-044523.671,NaN,NaN,NaN,102293092873892523,1049658000060517,397026776902759,0,12246887638605,10229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PS18kz,PSO J132531.079-142122.461,NaN,NaN,NaN,90772013794782874,1040248000047767,422246824852118,0,11265143992289,9077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SDSS-IISN17461,PSO J034305.548+001507.550,NaN,NaN,NaN,107690557731417905,943938000022176,365716465276576,0,9423182795338,10769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SN2016ezh,PSO J015804.744+005221.791,NaN,NaN,NaN,106950295197783171,927303000104953,363457312561658,0,8961418540888,10695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acqetxn,PSO J235011.541+191045.231,NaN,NaN,NaN,131013575480825509,1076778000026511,161254547154832,0,13215295323352,13101,...,1.0,11.96380,5.68306,3.76124,3.02015,NaN,NaN,0.0,NaN,NaN
ZTF21acqnurt,PSO J223925.671-022448.679,NaN,NaN,NaN,105103398569764184,1076249000094978,400995326718721,0,12661889120003,10510,...,1.0,15.28430,2.52730,1.53292,1.24170,2.55906,NaN,0.0,False,NaN
ZTF21acqoloi,PSO J231829.296-011726.224,NaN,NaN,NaN,106453496221281681,1082715000043887,401837140257649,0,12704687240487,10645,...,1.0,NaN,9.90200,2.44196,1.46597,3.27679,NaN,0.0,NaN,NaN


In [9]:
g_hosts_has_ps1_objid_df = g_hosts_df[~g_hosts_df.objID.isna()]
g_hosts_has_ps1_objid_df.index.rename("ztf_object_id", inplace=True)
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df[g_hosts_has_ps1_objid_df.index.str.startswith('ZTF')]
g_hosts_has_ps1_objid_df.to_csv('../dataframes/g_hosts_has_ps1_objid_df.csv')
g_hosts_has_ps1_objid_df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaaenu,PSO J014921.296+353104.780,NaN,NaN,NaN,150620273387282157,942081000036636,201335181971228,0,17342042780099,15062,...,1.0,13.51850,4.98918,4.267330,3.78907,9.400590,NaN,0.0,False,NaN
ZTF17aaaazhi,PSO J021351.579+333011.621,NaN,NaN,NaN,148200334649204503,936092000041298,201897822691666,0,17567891492385,14820,...,1.0,5.74010,1.27822,0.996905,1.49481,0.999151,NaN,0.0,False,NaN
ZTF17aaaazmz,PSO J020108.703+354438.836,NaN,NaN,NaN,150890302862623476,944990000044342,201356656815413,0,17342424965577,15089,...,1.0,4.73348,2.72390,3.379060,3.34216,4.355060,NaN,0.0,False,NaN
ZTF17aaabghb,PSO J050857.900-010153.780,NaN,NaN,NaN,106760772412722499,969245000054665,368151711765898,0,9372013840704,10676,...,1.0,5.70637,5.27978,7.873000,3.75759,2.645650,NaN,0.0,False,NaN
ZTF17aaabuab,PSO J032818.391-021919.410,NaN,NaN,NaN,105210520766363928,947850000033169,365261198754193,0,9429305171894,10521,...,1.0,3.08845,4.04116,4.027430,3.76517,11.518300,NaN,0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acqetxn,PSO J235011.541+191045.231,NaN,NaN,NaN,131013575480825509,1076778000026511,161254547154832,0,13215295323352,13101,...,1.0,11.96380,5.68306,3.761240,3.02015,NaN,NaN,0.0,NaN,NaN
ZTF21acqnurt,PSO J223925.671-022448.679,NaN,NaN,NaN,105103398569764184,1076249000094978,400995326718721,0,12661889120003,10510,...,1.0,15.28430,2.52730,1.532920,1.24170,2.559060,NaN,0.0,False,NaN
ZTF21acqoloi,PSO J231829.296-011726.224,NaN,NaN,NaN,106453496221281681,1082715000043887,401837140257649,0,12704687240487,10645,...,1.0,NaN,9.90200,2.441960,1.46597,3.276790,NaN,0.0,NaN,NaN


In [21]:
g_hosts_has_ps1_objid_df = g_hosts_has_ps1_objid_df.head()

In [34]:
g_hosts_has_ps1_objid_df

,objName,objAltName1,objAltName2,objAltName3,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,...,primaryDetection_y,gpetR90,rpetR90,ipetR90,zpetR90,ypetR90,NED_redshift_flag,hasSimbad,class,level_0
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaaenu,PSO J014921.296+353104.780,NaN,NaN,NaN,150620273387282157,942081000036636,201335181971228,0,17342042780099,15062,...,1.0,13.51850,4.98918,4.267330,3.78907,9.400590,NaN,0.0,False,NaN
ZTF17aaaazhi,PSO J021351.579+333011.621,NaN,NaN,NaN,148200334649204503,936092000041298,201897822691666,0,17567891492385,14820,...,1.0,5.74010,1.27822,0.996905,1.49481,0.999151,NaN,0.0,False,NaN
ZTF17aaaazmz,PSO J020108.703+354438.836,NaN,NaN,NaN,150890302862623476,944990000044342,201356656815413,0,17342424965577,15089,...,1.0,4.73348,2.72390,3.379060,3.34216,4.355060,NaN,0.0,False,NaN
ZTF17aaabghb,PSO J050857.900-010153.780,NaN,NaN,NaN,106760772412722499,969245000054665,368151711765898,0,9372013840704,10676,...,1.0,5.70637,5.27978,7.873000,3.75759,2.645650,NaN,0.0,False,NaN
ZTF17aaabuab,PSO J032818.391-021919.410,NaN,NaN,NaN,105210520766363928,947850000033169,365261198754193,0,9429305171894,10521,...,1.0,3.08845,4.04116,4.027430,3.76517,11.518300,NaN,0.0,False,NaN


In [46]:
%%time
for ztfid, ra, dec, ps1OID in zip(g_hosts_has_ps1_objid_df.index,
                           g_hosts_has_ps1_objid_df['raMean'], 
                           g_hosts_has_ps1_objid_df['decMean'], 
                           g_hosts_has_ps1_objid_df['objID']):    
    
    
    if dec >= 0: dec_int = int(math.floor(dec))
    else: dec_int = int(math.ceil(dec))
    
    # Open fits file corresponding to declination
    with fits.open(f'../ps1_psc/hlsp_ps1-psc_ps1_gpc1_{dec_int}_multi_v1_cat.fits') as F:
        objid_arr=F[1].data['objid']
        ps_score_arr=F[1].data['ps_score']
        
        try:
            ps_score = ps_score_arr[np.where(objid_arr == ps1OID)[0][0]]
            print(ps1OID, ps_score)
        except:
            print(f"{ps1OID} is not in this file. Set ps_score to -99")
            ps_score = -99

150620273387282157 0.025833333
148200334649204503 0.9295833
150890302862623476 0.0
106760772412722499 0.00375
105210520766363928 0.016416667


In [60]:
for ztfid, (ra_t, dec_t, ra_host, dec_host) in g_hosts_has_ps1_objid_df[['TransientRA', 'TransientDEC', 'raMean', 'decMean']].iterrows():
    print(ztfid)
    print(f'Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_t}+{dec_t}&filter=color')
    print(f'Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={ra_host}+{dec_host}&filter=color\n')

ZTF17aaaaenu
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.35194614323238+35.51434387930057&filter=color
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=27.33876949+35.5179556&filter=color

ZTF17aaaazhi
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=33.467575772476465+33.4979008400576&filter=color
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=33.46488349+33.50320637&filter=color

ZTF17aaaazmz
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=30.27832805514254+35.74524129697188&filter=color
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=30.28625948+35.74411702&filter=color

ZTF17aaabghb
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=77.23624207142889+-1.0266224000006332&filter=color
Host: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=77.24124535+-1.03161283&filter=color

ZTF17aaabuab
Transient: http://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos=52.0679571875008+-2.310774862502754&filter=color
Host: ht